In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math as m
import os
from scipy.stats import gaussian_kde

## Generate template using Infinity cube

### Infinity  cube code for RGB image

In [3]:
def infinte_cube(img):
    ''' Perform atomic operations on image similar to infinity cube. On performing these steps 'n' times, we can generate the biometric template.
        Input:
            img: The input image
        Output:
            returns the image after performing atomic operation once
    '''
    # Make width dimension even, if it is not
    if img.shape[1]%2 != 0:
        img = img[:, 0:img.shape[1]-1, :]    

    # Expand horizontally
    new_img = cv2.resize(img, (2*img.shape[1], img.shape[0]))

    # Make the height and width of even number
    if new_img.shape[0]%2 != 0:
        new_img = new_img[0:-2, :, :]
    if new_img.shape[1]%2 != 0:
        new_img = new_img[:, 0:-2, :]
#     plt.imshow(new_img)
#     plt.show()
#     new_img.shape

    # Split image vertically into top image and bottom image
    height, width, channels = new_img.shape
    half_height = height//2

    top_img, bottom_img = new_img[:half_height, :], new_img[half_height:, :]

#     plt.imshow(top_img)
#     plt.show()
#     plt.imshow(bottom_img)
#     plt.show()
#     top_img.shape, bottom_img.shape

    # Perform XOR operation between top and bottom section of images
    XOR_img = np.bitwise_xor(top_img, bottom_img)

#     plt.imshow(XOR_img)

    # Split image into 4 equal parts
    height, width, channels = XOR_img.shape
    quarter_width = width//4
    if width%4 != 0:
        quarter_width = int(width//4)
        XOR_img = XOR_img[:, 1:XOR_img.shape[1]-1, :]

    img1, img2, img3, img4 = XOR_img[:, 0:quarter_width], XOR_img[:, quarter_width: 2*quarter_width], XOR_img[:, 2*quarter_width:3*quarter_width], XOR_img[:, 3*quarter_width:]

#     plt.imshow(img1)
#     plt.show()
#     plt.imshow(img2)
#     plt.show()
#     plt.imshow(img3)
#     plt.show()
#     plt.imshow(img4)
#     plt.show()

    # Create new top and bottom images by merging middle two images and two outermost images respectively 
    top_img = cv2.hconcat([img2, img3])
    bottom_img = cv2.hconcat([img1, img4])

#     plt.imshow(top_img)
#     plt.show()
#     plt.imshow(bottom_img)
#     plt.show()
    
    # Form the image of originaly shape by concatenating the two images vertically
#     print(top_img.shape)
#     print(bottom_img.shape)
    new_img = cv2.vconcat([top_img, bottom_img])

#     plt.imshow(new_img)
#     new_img.shape
    
    # Expand vertically
    new_img = cv2.resize(new_img, (new_img.shape[1], 2*new_img.shape[0]))

#     plt.imshow(new_img)
#     plt.show()
#     new_img.shape

    # Split image horizontally into left image and right image
    height, width, channels = new_img.shape
    half_width = width//2

    left_img, right_img = new_img[:, :half_width], new_img[:, half_width:]

#     plt.imshow(left_img)
#     plt.show()
#     plt.imshow(right_img)
#     plt.show()
#     left_img.shape, right_img.shape

    # Perform XOR operation between left and right section of images
    XOR_img = np.bitwise_xor(left_img, right_img)

#     plt.imshow(XOR_img)

    # Split image into 4 equal parts
    height, width, channels = XOR_img.shape
    quarter_height = height//4

    img1, img2, img3, img4 = XOR_img[0:quarter_height, :], XOR_img[quarter_height:2*quarter_height, :], XOR_img[2*quarter_height:3*quarter_height, :], XOR_img[3*quarter_height:, :]

#     plt.imshow(img1)
#     plt.show()
#     plt.imshow(img2)
#     plt.show()
#     plt.imshow(img3)
#     plt.show()
#     plt.imshow(img4)
#     plt.show()

    # Create new left and right images by merging middle two images and two outermost images respectively 
    left_img = cv2.vconcat([img2, img3])
    right_img = cv2.vconcat([img1, img4])

#     plt.imshow(left_img)
#     plt.show()
#     plt.imshow(right_img)
#     plt.show()

    # Form the image of originaly shape by concatenating the two images vertically
    new_img = cv2.hconcat([left_img, right_img])
#     plt.imshow(new_img)
#     new_img.shape

    return new_img

# Generate template for a given image for given iterations
def inf_cube_with_itr(icwi_img, iterations):
    ''' Create a template for the given image with specified number of iterations
        Input:
            icwi_img: an image for which we want to generate the template
            iterations: the nuber of times we want to perform infinty cube transforamtions to generate the tempalte
        Output:
            returns the generated template for the given input image
    '''
    icwi_template = icwi_img.copy()
    for _ in range(iterations):
        icwi_template = infinte_cube(icwi_template)
    return icwi_template

## Create dataset

In [17]:
# Change the keys manually at location 1, location 2 to generate the dataset for different keys
# k1, k2, k3, k4, k5, k6
dataset_path = os.path.join(os.getcwd(), 'maze_dataset', 'k6') # location 1
save_path = os.path.join(os.getcwd(), 'PRP_with_ICT', 'k6') # location 2

for index, folder in enumerate(os.listdir(dataset_path), start=1):
    # print(index, folder)
    img_path1 = os.path.join(dataset_path, folder, '1.png')
    img_path2 = os.path.join(dataset_path, folder, '2.png')
    # print(img_path1, img_path2)
    
    user_name = f'user_{index}'
    
    img1 = cv2.imread(img_path1)
    img2 = cv2.imread(img_path2)
    
    new_img1 = infinte_cube(img1)
    new_img2 = infinte_cube(img2)

    save_dir = os.path.join(save_path, user_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    new_img_path1 = os.path.join(save_dir, '1.png')
    new_img_path2 = os.path.join(save_dir, '2.png')
    
    plt.imsave(new_img_path1, new_img1)
    plt.imsave(new_img_path2, new_img2)